In [1]:
from pypdf import PdfReader
from langchain_huggingface import HuggingFaceEmbeddings
# from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from pprint import pprint
from langchain_groq import ChatGroq
import os
# from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.llms import HuggingFaceTextGenInference
from langchain_huggingface import ChatHuggingFace
# import huggingface_hub
# from huggingface_hub import whoami
from langchain_community.document_loaders import PyPDFLoader
from langchain.schema import Document
from io import StringIO

In [2]:
# Read the API key from a file
with open("C:\Programming\doc reader chatbot\groq.txt", "r") as file:
    apiKey = file.read().strip()
# Set the API key as an environment variable
os.environ["GROQ_API_KEY"] = apiKey

<>:2: SyntaxWarning: invalid escape sequence '\P'
<>:2: SyntaxWarning: invalid escape sequence '\P'
C:\Users\mmibr\AppData\Local\Temp\ipykernel_14600\3610170531.py:2: SyntaxWarning: invalid escape sequence '\P'
  with open("C:\Programming\doc reader chatbot\groq.txt", "r") as file:


In [32]:
def processText(text):
    # Split the text into chunks using Langchain's RecursiveCharacterTextSplitter
    textSplitter = RecursiveCharacterTextSplitter(
        chunk_size = 256,
        chunk_overlap = 50,
        # length_function=len
        add_start_index=True
    )
    print(type(textSplitter))
    print(textSplitter)
    chunks = textSplitter.split_documents(text)
    print(type(chunks))
    print(len(chunks))
    # print(len(chunks[0]))
    print(type(chunks[0]))
    print(chunks[0])
    print(chunks[1])
    for chunk in chunks:
        print(f"Chunk metadata: {chunk.metadata}")
        print(f"Chunk length: {len(chunk.page_content)}")
    print(f"Split {len(text)} documents into {len(chunks)} chunks.")

    # Convert the chunks of text into embeddings to form a database
    embeddings = HuggingFaceEmbeddings()
    # db = Chroma.from_texts(chunks, embeddings)
    db = Chroma.from_documents(chunks, embeddings)

    return db

In [5]:
# TODO: Understand this function fully
def analyzeResults(query, results):
    prompt_template = """
    Answer the query based only on the given context:
    {context}
    ---
    Answer the query based on the above context: {query}
    """

    context = "\n\n---\n\n".join([doc.page_content for doc, _score in results])

    prompt = prompt_template.format(context = context, query = query)

    llm = ChatGroq()
    # llm = ChatGroq(model="llama3-8b-8192")
    # llm = ChatGroq(model = "llama-3.1-70b-versatile")
    response = llm.invoke(prompt)

    return response

In [12]:
file = "the-inspector-general--2.pdf"
# file = "The Prisoner of Zenda.txt"
# file = "Alice's Adventures in Wonderland.md"
# pdf = PdfReader(file)
loaderfile = PyPDFLoader(file)
# # loaderpdf = PyPDFLoader(pdf)
docs = loaderfile.load()
# # print(len(pdf.pages))
# print(type(file))
# print(type(pdf))
# print(type(loaderfile))
# print(type(docs))
# print()

# for doc in docs:
#     print(type(doc))
#     print(doc.metadata)
#     print(type(doc.metadata))
#     print(len(doc.metadata))
#     # print(type(doc.page_content))
#     print(len(doc.page_content))
#     print()

In [7]:
# text = ""
# for page in pdf.pages:
#     text += page.extract_text()
# print(len(text))

In [33]:
db = processText(docs)

<class 'langchain_text_splitters.character.RecursiveCharacterTextSplitter'>
<class 'list'>
718
<class 'langchain_core.documents.base.Document'>
page_content='The	Inspector-General
	
By
	
Nikolái	Vasí
lievi
ch	Gó
gol' metadata={'source': 'the-inspector-general--2.pdf', 'page': 0, 'start_index': 8}
page_content='CHARACTERS	OF	THE	PLAY
	
ANTON	ANTONOVICH	SKVOZNIK-DMUKHANOVSKY,	the
	
Governor.
ANNA	ANDREYEVNA,	his	wife.
MARYA	ANTONOVNA,	his	daughter.
LUKA	LUKICH	KHLOPOV,	the	Inspector	of	Schools.
His	Wife.
AMMOS	FIODOROVICH	LIAPKIN-TIAPKIN,	the	Judge.' metadata={'source': 'the-inspector-general--2.pdf', 'page': 1, 'start_index': 2}
Chunk metadata: {'source': 'the-inspector-general--2.pdf', 'page': 0, 'start_index': 8}
Chunk length: 57
Chunk metadata: {'source': 'the-inspector-general--2.pdf', 'page': 1, 'start_index': 2}
Chunk length: 241
Chunk metadata: {'source': 'the-inspector-general--2.pdf', 'page': 1, 'start_index': 198}
Chunk length: 226
Chunk metadata: {'source': 'the-inspector-gen

In [34]:
query = "What is the premise of the story?"

In [35]:
results = db.similarity_search_with_score(query)
pprint(results)

[(Document(metadata={'page': 29, 'source': 'the-inspector-general--2.pdf', 'start_index': 381}, page_content="MARYA.\tWell,\twell,\tmamma?\tIsn't\tit\tDobchinsky?\nANNA.\tYes,\tit\tis,\tI\tsee\tnow.\tWhy\tdo\tyou\targue\tabout\tit?\t[Calls\tthrough\tthe\nwindow.]\tHurry\tup,\tquick!\tYou're\tso\tslow.\tWell,\twhere\tare\tthey?\tWhat?\tSpeak\nfrom\twhere\tyou\tare.\tIt's\tall\tthe\tsame.\tWhat?\tHe\tis\tvery\tstrict?\tEh?\tAnd\thow\nabout\tmy\thusband?\t[Moves\taway\ta\tlittle\tfrom\tthe\twindow,\texasperated.]\tHe\tis\nso\tstupid.\tHe\twon't\tsay\ta\tword\tuntil\the\tis\tin\tthe\troom.\n\t\n\t\nSCENE\tII\nEnter\tDobchinsky."),
  1.0915517807006836),
 (Document(metadata={'page': 14, 'source': 'the-inspector-general--2.pdf', 'start_index': 461}, page_content="sake.\tThen\tthe\told\tfence\tnear\tthe\tbootmaker's\tmust\tbe\tpulled\tdown\tat\tonce\nand\ta\tpost\tstuck\tup\twith\ta\twhisp\tof\tstraw\tso\tas\tto\tlook\tlike\tgrading.\tThe\tmore\ndebris\tthere\tis\tthe\tmore\tit\twill\tshow\tt

In [36]:
response = analyzeResults(query, results)

In [38]:
pprint(response)
pprint(response.content)
print(response)
print(response.content)

AIMessage(content='Based on the given context, it is not possible to accurately describe the premise of the story. The text provided consists of several excerpts from a play, and while it provides some information about the characters (Marya, Anna, Dobchinsky) and their personalities, it does not give enough information about the plot or the overarching storyline. In order to determine the premise of the story, more context and information is needed.', response_metadata={'token_usage': {'completion_tokens': 94, 'prompt_tokens': 1007, 'total_tokens': 1101, 'completion_time': 0.150434075, 'prompt_time': 0.052708647, 'queue_time': 0.001152812000000003, 'total_time': 0.203142722}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-2cb31496-d4d7-4363-90f4-43037d27a00a-0', usage_metadata={'input_tokens': 1007, 'output_tokens': 94, 'total_tokens': 1101})
('Based on the given context, it is not possible to accurately d

In [118]:
sources = [doc.metadata.get("id", None) for doc, _score in results]
formattedResponse = f"Response: {response}\nSources: {sources}"
pprint(formattedResponse)

('Response: content="Based on the given context, the story appears to be set '
 'in a town where someone important is expected to arrive. The characters, '
 'Anna and Marya, are waiting near a window for this person to come. Anna '
 'expresses her frustration as she believes that the town is dirty and '
 'unkempt. She also mentions that she had relied on Dobchinsky but is now '
 'feeling let down.\\n\\nWhen Marya suggests that Dobchinsky is the person '
 'approaching, Anna initially disagrees but then realizes that it is indeed '
 'him. Anna calls Dobchinsky to hurry up and expresses her annoyance at his '
 'slow pace.\\n\\nScene II begins with Dobchinsky entering. Anna reprimands '
 'him for not being diligent and tells him that she had relied on him to take '
 'care of things.\\n\\nBased on this context, the story appears to be a '
 "critique of the town's lack of cleanliness and the inefficiency of its "
 'officials. The main character, Anna, is frustrated with the way things are '
